# Getting to know OOIPY
<img src="https://ooipy.readthedocs.io/en/latest/_images/OOIPY_Logo.png" width="100">

The documentation can be found [here](https://ooipy.readthedocs.io/en/latest/)

In [ ]:
import ooipy
from datetime import datetime
from IPython.display import Audio
import matplotlib.pyplot as plt
import numpy as np
import hvplot.xarray
import xarray as xr
import pandas as pd

### download broadband hydrophone data
- using `ooipy.get_acoustic_data` download a section of hydrophone data
    - data section

In [ ]:
starttime=datetime(2019,1,12,3)
endtime=datetime(2019,1,12,3,2)
node = 'LJ01C'

hdata = ooipy.get_acoustic_data(starttime, endtime, node, verbose=True)

   - downloads the relevant mseed files from the HTTP raw data server, and combines individual mseed files into a continuous data stream
   - If you are on Jupyter Hub, you can instead pull the data from the mounted ooi data directory
        - this allows you to pull in larger data (it's not going through the network)

In [ ]:
starttime=datetime(2019,1,12,3)
endtime=datetime(2019,1,12,4)
node = 'LJ01C'

hdata_jhub = ooipy.get_acoustic_data(starttime, endtime, node, verbose=True, jupyter_hub=True)

In [ ]:
len(hdata_jhub.data)*8*1e-9

### exploring the `ooipy.HydrophoneData` Object

#### obspy Trace methods
- wraps the `obspy.Trace` class (so it has all of the functionality of obspy traces)
   - plotting
   - decimation

In [ ]:
_ = hdata.plot()

In [ ]:
starttime=datetime(2019,1,12,3)
endtime=datetime(2019,1,12,3,2)
node = 'LJ01C'

hdata_temp = ooipy.get_acoustic_data(starttime, endtime, node, verbose=True)
hdata_temp.decimate(8)

In [ ]:
hdata_temp

#### estimate power spectral density

In [ ]:
psd = hdata.compute_psd_welch()

In [ ]:
psd.plot()
plt.xscale('log')

#### compute spectrograms
(Using PSD estimates)

In [ ]:
spec = hdata.compute_spectrogram()

In [ ]:
spec

In [ ]:
spec.hvplot.image(x='time', clim=(30,90), rasterize=True)

#### Playing with xarray a little bit
- let's take a little bit of time to familiarize ourselves with xarray

In [ ]:
spec_large = hdata_jhub.compute_spectrogram()

In [ ]:
spec_large

In [ ]:
spec_large.hvplot(rasterize=True)

- you can slice the dataset with coordinate values

In [ ]:
spec_slice = (
    spec_large
    .sel({'time':slice(pd.Timestamp('2019-01-12 T03:30:00'), pd.Timestamp('2019-01-12 T03:32:00'))})
    .sel({'frequency':slice(1000,10000)})
)

#### listen to audio in notebook

In [ ]:
Audio(hdata.data, rate=64e3)

#### save data to file

In [ ]:
hdata.save(file_format='nc', filename='LJ01C_20190112')

In [ ]:
xr.open_dataarray('LJ01C_20190112.nc').plot()

## Download Low Frequency Data with ooipy

In [ ]:
import ooipy
from datetime import datetime

In [ ]:
starttime = datetime(2018,5,1)
endtime = datetime(2018,5,1,1)
node='AXCC1'

hdata = ooipy.get_acoustic_data_LF(starttime, endtime, node, verbose=True)

In [ ]:
_ = hdata.plot()

In [ ]:
spec = hdata.compute_spectrogram()

In [ ]:
spec.plot(x='time', vmax=90, vmin=50)

In [ ]:
spec